In [1]:
%%html
<style>
table {
    display: inline-block
}
</style>

# Windows11：让普通PC升级为开发工作站
> 一般购买的原装Windows主机安装的是家庭版，缺少很多作为开发工作者需要的必要支持。因此需要对主机进行升权操作，成为开发工作人员的工作站。对于Windows10的用户也可以参考本文，具体功能入口点可能存在不同。


## 提纲

1. [操作系统提权](#1.操作系统提权)
2. [远程桌面访问](#2.远程桌面访问)
3. [远程SSH访问](#3.远程SSH访问)
4. [WSL](#4.WSL)
5. [挂载NFS共享目录](#5.挂载NFS共享目录)

网络唤醒 & 来电重启

## 参考资料

名称 | 说明 | 链接
:-- | :-- | :--
WSL文档 | 官网说明 | https://docs.microsoft.com/zh-cn/windows/wsl/
WSL代码 | github | https://github.com/microsoft/wsl
WSL文档 | windows10环境下安装说明 | https://docs.microsoft.com/zh-cn/windows/wsl/install-win10
查询在线WSL分发版本 | | https://raw.githubusercontent.com/microsoft/WSL/master/distributions/DistributionInfo.json
WSL离线安装明文档 | 包含下载链接 | https://docs.microsoft.com/en-us/windows/wsl/install-manual
WSL子系统原始镜像：Ubuntu
-| Ubuntu 22.04 LTS | https://aka.ms/wslubuntu2204
-| Ubuntu 20.04 | https://aka.ms/wslubuntu2004
-| Ubuntu 20.04 ARM | https://aka.ms/wslubuntu2004arm
-| Ubuntu 18.04 | https://aka.ms/wslubuntu1804
-| Ubuntu 18.04 ARM | https://aka.ms/wsl-ubuntu-1804-arm
-| Ubuntu 16.04 | https://aka.ms/wslubuntu1604
-| 默认 | https://aka.ms/wslubuntu
Debian
-| Debian | https://aka.ms/wsl-debian-gnulinux
Kali Linux
-| Kali | https://aka.ms/wsl-kali-linux-new
SUSE Linux
-| SUSE Linux Enterprise Server 12 | https://aka.ms/wsl-sles-12
-| SUSE Linux Enterprise Server 15 SP2 | https://aka.ms/wsl-SUSELinuxEnterpriseServer15SP2
-| SUSE Linux Enterprise Server 15 SP3 | https://aka.ms/wsl-SUSELinuxEnterpriseServer15SP3
openSUSE
-| openSUSE Tumbleweed | https://aka.ms/wsl-opensuse-tumbleweed
-| openSUSE 15.3 | https://aka.ms/wsl-opensuseleap15-3
-| openSUSE 15.2 | https://aka.ms/wsl-opensuseleap15-2
Oracle Linux
-| Oracle Linux 8.5 | https://aka.ms/wsl-oraclelinux-8-5
-| Oracle Linux 7.9 | https://aka.ms/wsl-oraclelinux-7-9
Fedora Remix
-| Fedora | https://github.com/WhitewaterFoundry/WSLFedoraRemix/releases/

# 1.操作系统提权

## 1.1.安装Win11
> 普通电脑升级Win11最常见的问题就是提示不满足Windows11的系统要求，即不支持TPM2.0。如下方法可以避免该问题。

### 1.Win10升级到Win11
> 通过[微软网站](https://www.microsoft.com/en-us/software-download/windows11)下载ISO文件。通过Windows的装载功能就可以将ISO作为一个光盘状态入系统，然后进行安装。如果安装过程提示Windows11的系统要求，可以在C盘找到`C:\WINDOWS.~BT\Sources\Panther\Appraiser_Data.ini`文件，将文件以管理身份打开，所有内容清空删除，再把文件属性改成只读即可。然后就可以按照正常安装过程进行安装。

### 2.通过安装盘安装Win11
> 在安装过程中，会提示`不符合安装基本要求`而无法完成下一步安装，这是由于win11要求TPM2.0导致在Esxi上无法直接安装。这个时候不需要重启安装过程，只需要回退到安装初始界面，通过修改注册表绕开检测：
<br> &emsp; 1.安装程序开始后，Shift+F10打开命令行界面，输入 regedit 打开注册表，定位到HKEY_LOCAL_MACHINE\SYSTEM\Setup；
<br> &emsp; 2.Setup→New→Key，创建名为“LabConfig”的项目，右键LabConfig →New→DWORD（32-bit）Value，名称为“BypassTPMCheck”，值为十六进制“00000001”,再创建名称为“BypassSecureBootCheck”，值为十六进制的“00000001”。
<br> &emsp; 3.存退出后重新点击“Install Now”就可以正常安装了。

### 3.跳过登录Microsoft账户
> 可以尝试以下几种方法：
<br> &emsp; 1.如果已经进入到安装过程，且电脑能联网：可以在联网界面输入一个不存在的邮箱地址和密码，系统会提示错误，之后就可以创建本地账户并跳过登录微软账户。
<br> &emsp; 2.如果电脑无法联网，可以尝试在安装过程前就按下Shift+F10（或Shift + Fn + F10）打开命令提示符，然后输入命令`oobe\bypassnro`；或者在命令提示符中输入taskmgr，打开任务管理器在“详细信息”找到OOBENetworkConnectionFlow.exe 右键单击，结束任务。

## 1.2.Win 10 & 11升级为专业版|企业版License
> 注意：Hyper-V仅适用于64位的 Windows 10/11 专业版、企业版、教育版和部分家庭版！一般购买的电脑、笔记本在出厂时都安装最低配置的家庭版本，不适合作为开发者使用，当然也无法安装Hyper-V。因此需要进行License的升级（不会修改原来电脑已经安装的软件）。

**步骤如下：**
1. 查看主机的Windows License
> <img style="width: 30%;display: inline-block" src="img/Windows规格.png"><img style="width: 30%;display: inline-block" src="img/windows激活.png">
<br> 开始 右键菜单 -> 系统 -> 关于 -> Windows规格 | 更改产品密钥或升级升级Windows。如图是已经升级到企业版的状态，如果是显示家庭版，就继续下面的步骤。


2. 以管理员身份打开CMD
> <img style="width: 30%;display: inline-block" src="img/打开windows的cmd.png">

3. 逐行输入如下的指令激活操作升级为64位专业版
````shell
slmgr.vbs /upk
# 企业版：slmgr /ipk NPPR9-FWDCX-D2C8J-H872K-2YT43，如下为专业版
slmgr /ipk W269N-WFGWX-YVC9B-4J6C9-T83GX
# KMS服务可以自己搭建，也可以修改端口号，比如：slmgr /skms htalker.freeddns.org:11688
slmgr /skms leedogood.duckdns.org
slmgr /ato
# 激活有效期为半年，到时间操作系统会自动到指定KMS服务上再次激活一次。
slmgr /xpr
````
> <img style="width: 30%;display: inline-block" src="img/win10成功激活.png">




4. 需要重启操作系统。

#### 注意：以上的操作过程全部一样，不同的在于License 序列号。Win10和Win11的序列号应该是通用的，如果不行可以更改后测试。

## 1.3.提权后的可选设置
> 有些设置并不建议，比如免密码登录。不过作为开发用的工作站，很多情况下是需要远程或者无人值守的情况下运行，有一些措施是需要根据自身情况进行选择。

### 1.升级到最新的Windows补丁
> 由于虚拟机类的软件与操作系统的关系比较紧密，一些安装方法不一定适合最新的软件。因此尽量升级到最新的补丁，并下载最新的软件版本用于安装。

### 2.打开开发人员模式
> 方法： 开始 -> 设置 -> 系统 -> 开发者选项：<b> 开发人员模式 开 </b>

### 3.打开传递优化
> 为了加快组件下载速度，同学们需要打开Windows的传递优化，方法：开始 右键菜单 -> 设置 -> 更新和安全 -> 传递优化 -> <b> 开 & 我本地网络上的电脑和Internet上的电脑 </b>

### 4.启用TLS v1.1 & v1.2
> win+r打开运行界面，输入control，打开控制面板，方法：网络与Internet -> Internet选项 -> 高级 -> 使用 TLS 1.0 & 1.1 & 1.2->

### 5.无密码登录Windows
> 登录Windows的过程需要输入用户名和密码，对于无人值守的服务器而言这个过程会导致一些服务在系统重启后就无法使用，比如：DockerDesktop就需要在控制台上启动才能使用。通过设置空密码和修改Windows注册表中的无密码登录项来实现。

* 方法一：netplwiz
> 设置无需账号密码登录本机，运行框输入：`netplwiz` 进入如下界面，将必须输入用户名和密码的多选框不选择，然后设置账号密码为空即可。
<img style="width: 50%;display: inline-block" src="img/Win10设置密码.png">

* 方法二：regedit
> 设置账号密码为空，然后运行框输入：`regedit`，找到 `计算机\HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows NT\CurrentVersion\PasswordLess\Device`，将键值 `DevicePasswordLessBuildVersion` = 0

### 6.win11桌面：网络、控制面板等
> 作为开发工作站，一些Windows的常用功能需要经常使用。我们可以将其放到桌面上。方法是：方法：开始 右键菜单 -> 个性化 -> 主题 -> 桌面图标设置。
<br> <img style="width: 50%;display: inline-block" src="img/win11桌面图标设置.jpg">

# 2.远程桌面访问

## 2.1.快速开始
### 1.启用步骤（略）
> 1、远程桌面有多种连接方式，包括WEB远程桌面（需要IIS支持）。2、可以修改默认端口号；3、通过账号设置来选择可以远程桌面的用户；4、通过防火墙设置网络过滤条件。

### 2.优缺点
> 图形化界面，操作简单；缺点是带宽、延时对效果影响大，其次对于个人电脑来说不能支持多用户操作。

## 2.2.远程桌面无密码登录
> 如果你的Windows是设置为无密码登录，那么远程桌面的默认安全策略是禁止访问的。通过设置本地安全策略组实现，运行框输入：`gpedit.msc` 进入设置界面。`计算机设置-- Windows 设置--安全设置--本地策略--安全选项`，找到：`帐户：使用空白密码的本地帐户只允许进行控制台登录`，并设置为：`已禁用`

# 3.远程SSH访问
> 远程桌面访问的带宽要求是很高的，如果在带宽有限而个人对命令行比较熟悉的情况下强烈推荐SSH远程访问。由于Winodws设计初衷是面向计算机水平较低的普通人员，因此较为注重GUI的操作，不过今年来Windows越来越向Linux的操作理念靠近，因此命令行完成大多数的任务问题不大。
<br> &emsp; Windows主机安装OpenSSH实现远程管理：Windows10/11的最新专业版已经支持通过SSH的方式来完成远程访问，这种方法的好处是获得Linux一样的远程管理能力。注意：OpenSSH可安装到 Windows 10 版本 1809 及更高版本上，因此注意要及时更新最新的补丁。

## 3.1.安装

1. 安装方法一
> 通过 设置 -> 系统 -> 可选功能：搜索ssh，然后安装 “OpenSSH 服务器”。如果没有找到可选功能可以：设置：查找设置输入 可选功能。

2. 安装方法二

````shell
#以管理员身份打开 PowerShell
Start-Service sshd
Set-Service -Name sshd -StartupType 'Automatic'
Get-NetFirewallRule -Name *ssh*
New-NetFirewallRule -Name sshd -DisplayName 'OpenSSH Server (sshd)' -Enabled True -Direction Inbound -Protocol TCP -Action Allow -LocalPort 22
net start sshd
net stop sshd
````

## 3.2.启停服务
> 判断ssh是否启动的最好方法是查看ssh的端口号是否开启，除此之外也可以通过服务进程来判断。

`netstat -an | find "22"`

1. 方法1
> 以管理员身份运行cmd,输入命令：services.msc，系统会打开后台服务管理界面。找到“OpenSSH SSH服务”的服务，可以进行管理。启动类型可以设置为“自动”，随主机启动而启动。

2. 方法2

````shell
#1. 以管理员身份打开 PowerShell/cmd
net start sshd
net stop sshd
net start ssh-agent
#2. 以管理员身份打开 PowerShell
start-service sshd
stop-service sshd
````

## 3.3.登录账号
> 使用ssh客户端工具，默认情况下账号使用Windows管理员账号和密码进行登录。当然也可以通过修改配置。

## 3.4.远程访问
1. 防火墙设置
> 找到 OpenSSH的条目进行配置管理

2. ssh设置
> 将监听端口改为:`0.0.0.0:22`

## 3.5.配置文件位置
> Windows下的OpenSSH的配置信息和Linux下几乎一样。只是配置文件所在位置不太一样。

1. OpenSSH默认安装文件夹：`%SystemRoot%\System32\OpenSSH`
2. OpenSSH服务端配置文件所在位置：`%programdata%\ssh\sshd_config`

## 3.6.免密码登录（通过部署证书）

1. 生成密钥
> 接下来需要在本地生成SSH密钥。打开“命令提示符”，输入“ssh-keygen -t rsa”，然后按照提示输入名称和密码，即可生成SSH密钥。生成完成后，在“C:Users你的用户名.ssh”目录下可以看到生成的密钥文件，分别为“id_rsa”和“id_rsa.pub”。其中“id_rsa”为私钥需要妥善保存。

2. 部署公钥 & 修改配置
> 无论是Linux公钥部署到“~/.ssh/authorized_keys”文件中，Windows系统部署到“%USERPROFILE%\.ssh\authorized_keys”，如果文件目录不存在需要手动创建。另外，文件内容中的一行为一个公钥钥匙，可以有多把公钥钥匙表示多个证书都可以免密码登录。

***注意***
> Windows新的OpenSSH服务版本将公钥部署位置修改了地方，具体查询：`%programdata%\ssh\sshd_config`中的内容，将修改为类似如下配置，重启SSH服务。

````shell
PubkeyAuthentication yes
# Match Group administrators
#       AuthorizedKeysFile __PROGRAMDATA__/ssh/administrators_authorized_keys
````

3. 免密码登录测试
> SSH客户端使用私钥进行登录，即在“命令提示符”中输入“ssh -i ~/.ssh/id_rsa username@ipaddress”，即可通过密钥登录到远程服务器。其中，username为远程服务器的用户名，ipaddress为远程服务器的IP地址。

## 3.7.Windows无密码
> 由于设置Windows无密码登录后，那么Windows安装的OpenSSH是需要使用Windows账号的，如何实现无密码登录OpenSSH？

* 方法一：采用证书登录方式（略）

````shell
# 启用证书登录
PubkeyAuthentication yes
AuthorizedKeysFile	.ssh/authorized_keys
````

* 方法二：如下步骤
> 1. 修改组策略(参考：2.2-2)<br>2. 修改OpenSSH的配置（%programdata%\ssh\sshd_config）如下：

````shell
# 将 no 改为 yes，允许空密码登录。
PermitEmptyPasswords yes

# 重启服务生效
services.msc
````

## 3.8.注意事项

1. win10请升级到预览版本（Pre-release）
> 通过OpenSSH登录到Windows宿主机后，运行wsl会提示：系统无法访问此文件。（The file cannot be accessed by the system. ） 微软已经确认该问题，并在预览版本中修复。目前release版本为：1.2.5仍然存在该问题：https://superuser.com/questions/1714736/cannot-run-wsl2-over-ssh-on-windows-11 。
通过wsl升级安装2.0以后的版本即可解决，方法：通过github官方网站下载wsl2.0预览版本，比如：https://github.com/microsoft/WSL/releases/tag/2.0.4 。然后安装升级即可解决。另外安装说明执行命令：`/usr/bin/wslinfo --networking-mode` 避免系统崩溃。

2. 避免wsl分发版本自动停止运行被系统回收
> 目前wsl的使用策略是当不使用的时候，会自动停止运行，这对于远程访问来说非常不便。为了避免wsl分发版本自动停止运行，可以有几种方法：1、将Windows设置为无密码自动登录，将将发行版本与DockerDesktop集成在一起，DockerDesktop配置为Windows登录后自动运行；2、将发行版本与DockerDesktop集成在一起，通过SSH远程运行：`"C:\Program Files\Docker\Docker\Docker Desktop.exe"`；3、OpenSSH后打开wsl -d 发行版本，缺点是终端不能退出。

3. 所有配置修改都要重启“OpenSSH SSH服务”

# 4.WSL
> Windows Subsystem for Linux（简称WSL）适用于 Linux 的 Windows 子系统可让开发人员按原样运行 GNU/Linux 环境 - 包括大多数命令行工具、实用工具和应用程序 - 且不会产生传统虚拟机或双启动设置的开销。 WSL 1 和 WSL 2 之间的主要区别在于，2.0在托管 VM 内使用实际的 Linux 内核、支持完整的系统调用兼容性以及跨 Linux 和 Windows 操作系统的性能。WSL 2相当于Linux的内核可以提供给WSL分发系统和Dockerdesktop所使用。

## 4.1.安装WSL

### 1.启用WSL功能组件
> 通过 [控制面板](#6.win11桌面：网络、控制面板等) -> 程序和功能 -> 启用或关闭Windows功能（左上角）中勾选：适合于Linux的Windows子系统（即WSL）

<img style="width: 30%;display: inline-block" src="img/打开windows的WSL组件.png">

**另外可以选择离线安装**
> Win11（https://files.xtalker.cn:8000/软件/wsl.2.1.5.0.x64.msi）
<br> Win10（https://files.xtalker.cn:8000/软件/wsl1.2.5x64.msi）
<br> 微软官方下载如上的安装包，安装过程很简单，略过。

### 2.WSL更新
> 此操作并不是更新WSL命令工具而是更新WSL内核。

````shell
wsl --update
--------------------------------
正在安装: 适用于 Linux 的 Windows 子系统
已安装 适用于 Linux 的 Windows 子系统。

wsl --version
--------------------------------
WSL 版本： 1.2.5.0
内核版本： 5.15.90.1  —— 注意核对内核版本号
WSLg 版本： 1.0.51
MSRDC 版本： 1.2.3770
Direct3D 版本： 1.608.2-61064218
DXCore 版本： 10.0.25131.1002-220531-1700.rs-onecore-base2-hyp
Windows 版本： 10.0.19045.3324

````
### 3.设置WSL 2

````shell
wsl --set-default-version 2
````

## 4.2.安装分发系统

### 1.WSL虚拟机映像资源查询
> 由于网络原因，很多WSL的在线下载资源是无法访问的。因此如果网络不通情况下可以考虑离线安装WSL虚拟机。

````shell
# 列表显示已经安装的WSL虚拟机实例
wsl -l
适用于 Linux 的 Windows 子系统分发:
Ubuntu-20.04 (默认)
# 查询可用的Linux安装版本列表：-o 参数是显示在线的WSL虚拟机映像。是网络问题，可以通过设置系统代理以及通过浏览器访问如下链接来解决。
wsl -l -o
无法从“https://raw.githubusercontent.com/microsoft/WSL/master/distributions/DistributionInfo.json”中提取列表分发。安全频道支持出错
Error code: Wsl/0x80072f7d
````

### 2.在线查询&安装

````shell
1. 查询可用的Linux安装版本列表
wsl -l -o
2. 设置WSL版本为 2
wsl --set-default-version 2
3. 我们选择安装  Ubuntu-20.04
wsl --install -d Ubuntu-20.04
正在安装: 虚拟机平台
已安装 虚拟机平台。
正在安装: 适用于 Linux 的 Windows 子系统
已安装 适用于 Linux 的 Windows 子系统。
正在下载: WSL 内核
正在安装: WSL 内核
已安装 WSL 内核。
正在下载: Ubuntu
````

### 3.离线安装(由于在线安装依赖于网络带宽，推荐使用离线安装)

* 本地默认安装
选择长期支持版本：Ubuntu2204LTS-230518_x64.appx
点击文件右键菜单选择安装即可一步步完成。

* 选择安装位置安装
> 将文件后缀 `.appx` 改为.zip，并解压到想要安装到的路径下(有足够的剩余空间)。解压后的目录中 ，双击ubuntu2004.exe，等待一会，它会自动在此目录下安装好ubuntu。

### 4.首次启动
> 从windows开始菜单找到Ubuntu，进入后设置初始账号和密码（lab / lab123）就完成了所有安装。

## 4.3.WSL的使用

### 1.常用指令

````shell
# 管理员模式打开PowerShell 或 Windows 命令提示符
# 帮助
wsl --help
# 设置wsl版本为2.0
wsl --set-default-version 2
# 查看所有已经安装的WSL虚拟机情况
wsl -l -v
  NAME                   STATE           VERSION
* Ubuntu-20.04           Running         2
  docker-desktop-data    Running         2
  docker-desktop         Running         2
# 进入默认的分发子系统（如上*代表默认的WSL）
wsl 或者 bash
# 将虚拟机设置为默认值
wsl -s 名称
# 运行指定分发系统
wsl -d 名称
# 终止指定的分发系统
wsl -t 名称
# 卸载分发系统
wsl --unregister 名称
# 关闭所有正在运行的分发系统
wsl --shutdown
# 迁移分发系统
wsl --export 名称 文件导出路径
# 导入新的分发系统（wsl 2，导入分发系统名称或者文件路径等等）
wsl --import 名称 目标路径 文件路径 --version 2
````

### 2.分发系统和宿主机文件互访

1. 宿主机访问分发系统文件
> 在文件资源管理器中输入: `\\wsl.localhost`  + WSL实例名 或者 `\\wsl$ 注意结尾加$` + WSL实例名。

2. 访问宿主机文件
> 在虚拟机命令提示符下输入：`cd /mnt/盘符` 例如进入D盘：`cd /mnt/d`

3. 文件的权限
> 从Windows资源管理器访问WSL的时候，只有登录Home目录的修改权限，其它目录只能查看。比如：/home/lab目录下可以修改创建文件。

## 4.4.WSL2支持systemd
> 注意该特性是针对较新的Win10和Win11，需要及时更新你的补丁。另外要注意更新WSL内核。如果不支持，那么可以参考：https://superuser.com/questions/1343558/how-to-make-wsl-run-services-at-startup/1685207#1685207 实现自动启动一些服务，如SSH。

````shell
# 为发行版开启 systemd
sudo sh -c 'echo "[boot]\nsystemd=true" >> /etc/wsl.conf'
# 注意一些最新的发行版已经支持，方法就是查询 /etc/wsl.conf 文件中的内容
cat /etc/wsl.conf
---------------------------
[boot]
systemd=true
# 如果支持就可以使用 Systemd 相关的服务和命令了
systemctl status ssh
sudo systemctl enable ssh
sudo systemctl restart ssh
````

## 4.5.WSL自动关闭实例的问题
> WSL2会默认关闭不使用的实例，当你关闭了WSL的Console后，实例会自动关闭。解决办法：

1. Dockerdesktop集成
> 如果安装有Dockerdesktop，则可以通过 Setting -> Resources -> WSL Integeration: 将需要长期运行的实例集成进来，则只要Dockerdesktop不关闭则WSL发行版实例也不会关闭。

2. 通过wscript.shell

````shell
set ws=wscript.CreateObject("wscript.shell")
ws.run "wsl -d 发行版本名字", 0
````

## 4.6.WSL资源限制
> 通过配置 .wslconfig 文件来限制 WSL 的资源占用。 WSL 会默认占用50%内存，最大8GB，使用所有CPU线程。如下配置是限制4GB，2个线程。

````shell
notepad %USERPROFILE%\.wslconfig

[wsl2]
memory=4GB 
processors=2
````

## 4.7.网络访问

### 1.WSL网络原理
> 宿主机IP地址为：172.28.17.120/24，WSL虚拟网卡地址为：172.20.48.1/20。WSL发行版本实例的地址为：172.20.60.234/20。WSL虚拟网卡是所有WSL实例的网关，与实例处于同一个局域网地址中。其IP地址由Hyper-V的软交换机的DHCP服务进行分配。由于网络：172.20.48.0/20无法直接与外网通信，因此WSL实例是无法被宿主机以外的设备访问的。因此需要和Docker类似进行端口映射，不过默认情况下实例内的端口无论环回地址监听还是网卡上监听，WSL都会默认在宿主机的127.0.0.1地址上监听，可以通过宿主机的127.0.0.1地址进行访问。

````shell
# 发行版实例的地址
ifconfig
eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1280
        inet 172.20.60.234  netmask 255.255.240.0  broadcast 172.20.63.255
        inet6 fe80::215:5dff:fe53:4da6  prefixlen 64  scopeid 0x20<link>
        ether 00:15:5d:53:4d:a6  txqueuelen 1000  (Ethernet)
        RX packets 10559  bytes 6952471 (6.9 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 4067  bytes 544560 (544.5 KB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

# 查询发行版实例的网关
route -n
Kernel IP routing table
Destination     Gateway         Genmask         Flags Metric Ref    Use Iface
0.0.0.0         172.20.48.1     0.0.0.0         UG    0      0        0 eth0
172.20.48.0     0.0.0.0         255.255.240.0   U     0      0        0 eth0

ip route show
default via 172.20.48.1 dev eth0 proto kernel 
172.20.48.0/20 dev eth0 proto kernel scope link src 172.20.60.234 

# 查询发行版实例SSH的端口号
sudo netstat -tunpl | grep 22
tcp        0      0 127.0.0.1:22            0.0.0.0:*               LISTEN      2979/sshd: /usr/sbi

# 查询宿主机上端口号
netstat -an | find "22"
  TCP    127.0.0.1:22           0.0.0.0:0              LISTENING

````

### 2.网络端口映射
> 官网说明：https://learn.microsoft.com/zh-cn/windows/wsl/networking <br> netsh说明：https://learn.microsoft.com/zh-cn/windows-server/networking/technologies/netsh/netsh-interface-portproxy <br><br> 以SSH服务为例 <br> 宿主机的2222端口：127.0.0.1: 22（环回地址）；<br> 宿主机的2233端口：172.20.60.234: 22（WSL实例网卡地址）。<br> <br> 注意：宿主机需要设置防火墙外网才能访问。 <br> <br> 如果端口号一样，系统首先响应的是环回地址。

````shell
# 宿主机的2222端口号映射到127.0.0.1:22
netsh interface portproxy set v4tov4 listenport=2222 listenaddress=0.0.0.0 connectport=22 connectaddress=127.0.0.1
# 宿主机的2233端口号映射到WSL发行版实例网卡地址：172.20.60.234
netsh interface portproxy add v4tov4 listenport=2233 listenaddress=0.0.0.0 connectport=22 connectaddress=172.20.60.234
# 显示所有 portproxy 参数
netsh interface portproxy show all

侦听 ipv4:                 连接到 ipv4:

地址            端口        地址            端口
--------------- ----------  --------------- ----------
0.0.0.0         2222        127.0.0.1       22
0.0.0.0         2233        172.20.60.234   22
# 宿主机需要设置防火墙端口号，就可以访问2222和2233端口号，都是访问同一个WSL实例。
# 删除
netsh interface portproxy delete v4tov4 listenport=2222 listenaddress=0.0.0.0
netsh interface portproxy delete v4tov4 listenport=2233 listenaddress=0.0.0.0
````



# 4.8.WSL部分发行版

### 1.Ubuntu 20.04
> 除了Windows主机可以通过安装OpenSSH服务实现远程管理以外，WSL也可以。这里以Ubuntu20.04版本的WSL为例，说明如何安装国内更新源以及安装并启动OpenSSH的过程，以实现远程管理。

1. sudo
> 进入Linux命令行终端后，默认登录是WSL Linux安装过程中设置的普通账号。通过输入：`sudo 命令`回车的方式执行系统管理员命令，如果`sudo`未安装,则可以通过`su`命令升级权限为root用户，然后执行`apt update` `apt install sudo`。

2. 普通用户登录后执行命令如下：

````shell
#先安装认证中心，否则更换国内安装源报错。
sudo apt update
sudo apt install ca-certificates apt-transport-https apt-utils
#备份 sources.list
sudo mv /etc/apt/sources.list /etc/apt/sources.list.backup
#更新国内源配置
echo "#默认注释了源码镜像以提高 apt update 速度，如有需要可自行取消注释" | sudo tee /etc/apt/sources.list
echo "deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ focal main restricted universe multiverse" | sudo tee -a /etc/apt/sources.list
echo "#deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ focal main restricted universe multiverse" | sudo tee -a /etc/apt/sources.list
echo "deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ focal-updates main restricted universe multiverse" | sudo tee -a /etc/apt/sources.list
echo "#deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ focal-updates main restricted universe multiverse" | sudo tee -a /etc/apt/sources.list
echo "deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ focal-backports main restricted universe multiverse" | sudo tee -a /etc/apt/sources.list
echo "#deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ focal-backports main restricted universe multiverse" | sudo tee -a /etc/apt/sources.list
echo "deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ focal-security main restricted universe multiverse" | sudo tee -a /etc/apt/sources.list
echo "#deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ focal-security main restricted universe multiverse" | sudo tee -a /etc/apt/sources.list
#等待安装资源的索引更新
sudo apt update
#安装基本工具，配置时区和中文环境
sudo apt install sudo net-tools inetutils-ping vim screen tzdata locales language-pack-zh-hans
sudo ln -sf /usr/share/zoneinfo/Asia/Shanghai /etc/localtime 
echo 'LANG="zh_CN.UTF-8"' | sudo tee -a /etc/default/locale
echo 'LANGUAGE="zh_CN:zh:en_US:en"' | sudo tee -a  /etc/default/locale 
echo 'LC_NUMERIC="zh_CN.UTF-8"' | sudo tee -a  /etc/default/locale 
echo 'LC_TIME="zh_CN.UTF-8"' | sudo tee -a  /etc/default/locale
echo 'LC_MONETARY="zh_CN.UTF-8"' | sudo tee -a  /etc/default/locale
echo 'LC_PAPER="zh_CN.UTF-8"' | sudo tee -a  /etc/default/locale
echo 'LC_IDENTIFICATION="zh_CN.UTF-8"' | sudo tee -a  /etc/default/locale
echo 'LC_NAME="zh_CN.UTF-8"' | sudo tee -a  /etc/default/locale
echo 'LC_ADDRESS="zh_CN.UTF-8"' | sudo tee -a  /etc/default/locale
echo 'LC_TELEPHONE="zh_CN.UTF-8"' | sudo tee -a  /etc/default/locale
echo 'LC_MEASUREMENT="zh_CN.UTF-8"\n' | sudo tee -a  /etc/default/locale
#安装设置 openssh 环境.
sudo apt install openssh-server
#设置root账号密码
sudo passwd root
#解锁root远程登录权限
sudo passwd --unlock root
#这样就解锁root账号了。
#设置SSH服务，允许root远程访问"
echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
echo "PubkeyAuthentication yes" >> /etc/ssh/sshd_config
#启动ssh服务
service ssh restart
````


### 2.Ubuntu 22.04

1. 更改更新源配置文件：/etc/apt/sources.list

````shell
sudo mv /etc/apt/sources.list /etc/apt/sources.list.backup
sudo vim /etc/apt/sources.list
-----------------------------------------------------------------
# 默认注释了源码镜像以提高 apt update 速度，如有需要可自行取消注释
deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy main restricted universe multiverse
deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy main restricted universe multiverse
deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-updates main restricted universe multiverse
deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-updates main restricted universe multiverse
deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-backports main restricted universe multiverse
deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-backports main restricted universe multiverse

# deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-security main restricted universe multiverse
# deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-security main restricted universe multiverse

deb http://security.ubuntu.com/ubuntu/ jammy-security main restricted universe multiverse
deb-src http://security.ubuntu.com/ubuntu/ jammy-security main restricted universe multiverse

# 预发布软件源，不建议启用
deb https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-proposed main restricted universe multiverse
deb-src https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ jammy-proposed main restricted universe multiverse
````
2. 安装必要的软件

````shell
sudo apt install --no-install-recommends --no-install-suggests -y net-tools inetutils-ping dialog curl wget sudo vim screen init  systemd tzdata locales
````

3. 配置中文环境

````shell
# 选择语言包
sudo sed -i -e "s/# en_US.UTF-8 UTF-8/en_US.UTF-8 UTF-8/" /etc/locale.gen
sudo sed -i -e "s/# zh_CN.UTF-8 UTF-8/zh_CN.UTF-8 UTF-8/" /etc/locale.gen
# 激活语言包
sudo dpkg-reconfigure --frontend=noninteractive locales
# 设置默认语言环境为中文
sudo update-locale LANG=zh_CN.UTF-8
````

4. 安装OpenSSH

````shell
sudo apt install --no-install-recommends --no-install-suggests -y openssh-server

````

# 5.挂载NFS共享目录
> NFS服务器端：群晖NAS

### 1.安装NFS客户端
> 打开“控制面板” > “程序” > “启用或关闭Windows功能”-> NFS服务：勾选“NFS客户端”和“管理工具”，然后点击确定。等待安装完成，窗口会自动关闭。

### 2.映射网络驱动器（不推荐）
> 资源管理 | 桌面 -> 网络：鼠标右键 -> 映射网络驱动器：
<br> &emsp; 驱动器：Windows本机上所映射的磁盘。
<br> &emsp; 文件夹：NFS访问地址（`\\IP|主机名\共享文件夹\子路径`，比如：`\\172.28.17.200\volume1\data_nfs`）
<br> &emsp; **——技巧提示：**可以输入 `\\IP|主机名\共享文件夹`，点击浏览就可以查看该服务器下所有的NFS共享目录。注意如果共享文件夹输入的是SMB共享目录，可能结果不太一样。因为Windows映射网络驱动器是支持多种网络协议的，其中包括NFS和SMB等。
<br> &emsp; 登录时重新连接：选择是否重启电脑时自动连接。

### 3.挂载NFS共享（命令行方式）
> 打开命令提示符（按下Win + R，输入cmd并回车）。如果需要定时任务或者开机启动，则使用`计算机管理` -> `系统工具` -> `任务计划管理`：创建合适的计划任务即可。

```shell
# 帮助信息
mount -h
...

# 显示远程NFS服务器的共享目录
showmount -e 172.28.17.200
导出列表在 172.28.17.200:
/volume1/data_nfs                  *

# 挂载命令
mount \\172.28.17.200\volume1\data_nfs\nuc8_esxi e: 
e: 现已成功连接到 \\172.28.17.200\volume1\data_nfs\nuc8_esxi

# 带参数
mount -o nolock -o timeout=60 \\172.28.17.200\volume1\data_nfs\nuc8_esxi e: 

# 显示当前挂载列表
mount

本地    远程                                 属性
-------------------------------------------------------------------------------
e:       \\172.28.17.200\volume1\data_nfs\nuc8~ UID=0, GID=0
                                                rsize=131072, wsize=131072
                                                mount=soft, timeout=0.8
                                                retry=1, locking=yes
                                                fileaccess=755, lang=GB2312-80
                                                casesensitive=no
                                                sec=sys

# 卸载命令
umount e:
```

### 4.用户组设置（无访问权限问题）
> `Windows`键 + R键：输入 regedit 打开注册表，定位到[HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\ClientForNFS\CurrentVersion\Default]；
<br> Setup→New→DWORD：
<br> &emsp; 用户ID： AnonymousUid DWORD = 0
<br> &emsp; 组ID： AnonymousGid DWORD = 0
<br> &emsp; 0即为root，重启系统生效
<br> &emsp; 登录到NFS服务器端查看客户端创建的文件权限，发现确实是`root`用户。

### 5.Locking默认参数（file handle error）
> `Windows`键 + R键：输入 regedit 打开注册表，定位到[HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\ClientForNFS\CurrentVersion\Users\Default\Mount]；
<br> Setup→New→DWORD：
<br> &emsp; Locking DWORD = 1
<br> &emsp; 重启系统生效